# Code pour reformater les vidéos au même nombre de frames pour faciliter l'extraction et le traitement

# 1. Imports

In [131]:
import cv2
import os
import numpy as np
from os.path import isfile, join
from moviepy.editor import VideoFileClip

# 2. Paramétrer les dossiers

Paramètres de bases

In [132]:
path = '../mots/' #path vers le dossier des mots
data_path = 'coords_npy/'
mots=np.array(['adresse','affaire']) #choix des mots
nb_videos=20 #nb de vidéos par mot

Calculer le nombre d'images moyen par vidéo pour avoir la référence

In [133]:
count=0
fichier =list([])
for mot in mots:
    fichier1=[]
    i=0
    for f in os.listdir(path):
        if i<nb_videos:
            file_name, file_ext = os.path.splitext(f)
            testmot = file_name.split('-')
            if (testmot[0]==mot):
                fichier1 += [path + file_name + file_ext]
                i+=1
    fichier.append(fichier1)
rang_mot=0
for mot in mots:
    for f in fichier[rang_mot]:
        cap = cv2.VideoCapture(f)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        count=count+length
    rang_mot+=1
moy=count/(len(fichier)*nb_videos)
nb_frame=int(moy)#nombre d'images par vidéos
print(nb_frame)

20


Création des dossiers pour le stockage des images & vidéos

In [134]:
for mot in mots: 
    for vid in range(nb_videos):
        try: 
            os.makedirs(os.path.join(data_path, mot, str(vid)))
        except:
            pass

# 3. Code

Fonction intermédiaire pour plus tard calculer le framerate nécessaire à l'obtention du bon nombre de frames

In [135]:
def duree_et_longueurCV(video):

    duree = video.get(cv2.CAP_PROP_POS_MSEC)
    longueur = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return duree, longueur
def dureeMPY(file):
    clip = VideoFileClip(file)
    return clip.duration

Fonction pour stocker une image

In [136]:
def getFrame(vidcap,sec,rang_video,count):
            vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
            hasFrames,image = vidcap.read()
            if hasFrames:
                if count<10:
                    numero = '0' + str(count)
                else :
                    numero = str(count)
                cv2.imwrite(data_path+mot+'/'+str(rang_video)+'/image'+numero+".jpg", image)     # save frame as JPG file
            return hasFrames

Fonction pour trouver le bon framerate afin d'extraire le bon nombre d'images

In [137]:
def load_video_and_find_framerate(fichier,nb_frame):
    video = cv2.VideoCapture(f)
    duree = dureeMPY(fichier)
    fr=video.get(cv2.CAP_PROP_FPS)
    if fr!=25:
        video.set(cv2.CAP_PROP_FPS, 25)
        duree = (25*duree)/fr
        fr=25
    framerate = (duree/fr)*(fr/nb_frame)
    #print(fr,duree,framerate)
    return video, framerate

Code pour stocker automatiquement toutes les images des mots nous intéressants

In [138]:
rang_mot=0
for mot in mots:
    #print(mot)
    rang_video=0
    for f in fichier[rang_mot]:
        #print(f)
        sec = 0
        count=1
        vidcap, framerate = load_video_and_find_framerate(f,nb_frame)
        success = getFrame(vidcap,sec,rang_video,count)
        while success:
            count = count + 1
            sec = sec + framerate
            success = getFrame(vidcap,sec,rang_video,count)
        #print(len(os.listdir(data_path+mot+'/'+str(rang_video)+'/')))
        rang_video+=1
    rang_mot+=1

# Analyser les images

### Préliminaires (mêmes fonctions que dans export_coord_v2)

In [139]:
import mediapipe as mp 
mp_drawing = mp.solutions.drawing_utils  # Drawing helpers
mp_holistic = mp.solutions.holistic  # Mediapipe Solutions

def mediapipe_detection(image, model): #Fonction pour que mediapipe puisse détecter (car il ne travaille pas dans le même système de couleur que OpenCV)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
                            
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    #return np.concatenate([pose, face, lh, rh])
    return np.concatenate([pose, lh, rh])

Vérification

In [140]:
print(nb_videos,nb_frame,data_path)

20 20 coords_npy/


### Code

Fonction retournant la liste des fichiers d'un dossier

In [141]:
def listfichier(path):
    L = os.listdir(path)
    L1 = L
    i=0
    for j in L:
        L1[i]  = path + L[i]
        i+=1
    #print(L)
    return L

In [142]:
rang_mot = 0
rang_video = 0
rang_frame=0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    mots = listfichier(data_path)
    for mot in mots:
        dossiers=listfichier(mot + '/')
        for d in dossiers:
            fichiers=listfichier(d + '/')
            for f in fichiers:
                cap = cv2.VideoCapture(f)
                ret, frame = cap.read()
                if ret :
                    image, results = mediapipe_detection(frame, holistic)

                    draw_styled_landmarks(image, results)

                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(mot, str(rang_video), str(rang_frame))
                    np.save(npy_path, keypoints)
                    
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                else :
                    break
                rang_frame+=1
                cap.release()
                cv2.destroyAllWindows()
            rang_frame=0
            rang_video+=1
        rang_video=0
        rang_mot+=1

Supprimer les images

In [143]:
for mot in mots:
        dossiers=listfichier(mot + '/')
        for d in dossiers:
            fichiers=listfichier(d + '/')
            for f in fichiers:
                if f[-1]=='g':
                    os.remove(f) #supprimer l'image